# pyXSIM Example

To show how to make a set of photons from a 3D dataset using pyXSIM and yt for 
reading into SOXS, we'll look at is that of thermal emission from a galaxy 
cluster. In this case, the gas in the core of the cluster is "sloshing" in 
the center, producing spiral-shaped cold fronts. The dataset we want to use 
for this example is available for download from the [yt Project](http://yt-project.org) 
at [this link](http://yt-project.org/data/GasSloshing.tar.gz). 

First, import our necessary modules:

In [ ]:
import yt
import pyxsim
import soxs

Next, we `load` the dataset with yt (this dataset does not have species fields, 
so we specify that the gas is fully ionized in this case so that the emission measure
field can be computed correctly):

In [ ]:
ds = yt.load(
    "GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150", default_species_fields="ionized"
)

Let's use yt to take a slice of density and temperature through the center of 
the dataset so we can see what we're looking at: 

In [ ]:
slc = yt.SlicePlot(
    ds, "z", [("gas", "density"), ("gas", "temperature")], width=(1.0, "Mpc")
)
slc.show()

Ok, sloshing gas as advertised. Next, we'll create a sphere object to serve as 
a source for the photons. Place it at the center of the domain with `"c"`, and 
use a radius of 500 kpc:

In [ ]:
sp = ds.sphere("c", (0.5, "Mpc"))

Now, we need to set up the emission model for our source. We said we were going 
to look at the thermal emission from the hot plasma, so we'll do that here by 
using `ThermalSourceModel`. The first four arguments are the name of the underlying 
spectral model, the maximum and minimum energies, and the number of bins in the 
spectrum. We've chosen these numbers so that the spectrum has an energy resolution 
of about 1 eV. Setting `thermal_broad=True` turns on thermal broadening. This 
simulation does not include metallicity, so we'll do something simple and say 
that it uses the above spectral model and the metallicity is a constant $Z = 0.3~Z_\odot$:

In [ ]:
source_model = pyxsim.CIESourceModel(
    "apec", 0.5, 9.0, 9000, thermal_broad=True, Zmet=0.3
)

We're almost ready to go to generate the photons from this source, but first 
we should decide what our redshift, collecting area, and exposure time should 
be. Let's pick big numbers, because remember the point of this first step is 
to create a Monte-Carlo sample from which to draw smaller sub-samples for mock 
observations. Note these are all (value, unit) tuples:

In [ ]:
exp_time = (300.0, "ks")  # exposure time
area = (3.0, "m**2")  # collecting area
redshift = 0.2

So, that's everything--let's create the photons!

In [ ]:
n_photons, n_cells = pyxsim.make_photons(
    "my_photons", sp, redshift, area, exp_time, source_model
)

Ok, that was easy. Now we have a photon list that we can use to create events, 
using the `project_photons()` function. To be realistic, we're going to want to assume 
foreground Galactic absorption, using the "TBabs" absorption model and assuming a 
foreground absorption column of $N_H = 4 \times 10^{20}~{\rm cm}^{-2}$. Here we'll 
just do a simple projection along the z-axis, reducing the exposure time, and 
centering the photons at RA, Dec = (30, 45) degrees:

In [ ]:
n_events = pyxsim.project_photons(
    "my_photons", "my_events", "z", (30.0, 45.0), absorb_model="tbabs", nH=0.04
)

We can then use this event list that we wrote as an input to the instrument simulator in SOXS. We'll use a smaller exposure time (100 ks instead of 500 ks), and observe it with the Lynx calorimeter:

In [ ]:
soxs.instrument_simulator(
    "my_events.h5",
    "evt.fits",
    (100.0, "ks"),
    "lynx_lxm",
    [30.0, 45.0],
    overwrite=True,
)

We can use the `write_image()` function in SOXS to bin the events into an image 
and write them to a file, restricting the energies between 0.5 and 2.0 keV:

In [ ]:
soxs.write_image("evt.fits", "img.fits", emin=0.5, emax=2.0, overwrite=True)

We can show the resulting image:

In [ ]:
fig, ax = soxs.plot_image(
    "img.fits", stretch="sqrt", cmap="afmhot", vmax=1000.0, width=0.05
)

We can also bin the events into a spectrum using `write_spectrum()` and write 
the spectrum to disk:

In [ ]:
soxs.write_spectrum("evt.fits", "evt.pha", overwrite=True)

and plot the spectrum using `plot_spectrum()`:

In [ ]:
fig, ax = soxs.plot_spectrum("evt.pha", xmin=0.5, xmax=7.0)

Let's zoom into the region of the spectrum around the iron line to look at 
the detailed structure afforded by the resolution of the calorimeter:

In [ ]:
ax.set_xlim(5.4, 5.7)
fig